# Financial Dataset with stock market Analysis from(2014-2018)

Flights analysis is going to be performed as follows:

1. PySpark **environment setup**
2. Data source and **Spark data abstraction** (DataFrame) **set up**
3. Data set **metadata analysis**: Display schema and size of the DataFrame
4. Profiling the columns being considered for our analysis:
 1. **Important columns:** Revenue, Revenue Growth, Net Profit Margin, Total liabilities, Total assets
 2. **Creating new columns**
 3. **Top 5 stocks on the basis of different measures**
5. Merging datasets: Financial dataset and actual stock
    1. Obtaining the data with date and obtain the prices after the financial data has been published
    2. Obtaining the increase in stock prices over years
    3. Developing analysis and categorized dataframes
6. Answer Business questions
    1. How many stocks have low and moderate risk, top quartile in revenue, net profit, and bottom two quartiles in terms of    price per each category?
    2. Which stocks are low risk, best performing and a good buy?
    3. Which low to moderate risk stocks to buy to play the trend?
    4. Which stocks to avoid ?
    
## 1. PySpark environment setup

In [1]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## 2. Data source and Spark data abstraction (DataFrame) setup

In [2]:
fdata2014 = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("Financial data/2014_Financial_Data.csv") 
fdata2015 = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("Financial data/2015_Financial_Data.csv") 
fdata2016 = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("Financial data/2016_Financial_Data.csv") 
fdata2017 = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("Financial data/2017_Financial_Data.csv") 
fdata2018 = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("Financial data/2018_Financial_Data.csv") 
stockprices = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("Financial data/all_stocks_5yr.csv")



## 3. Data set metadata analysis
###  Display schema and size of the DataFrame

In [3]:
from IPython.display import display, Markdown

fdata2014.printSchema()
display(Markdown("This DataFrame has **%d rows**." % fdata2014.count()))

root
 |-- _c0: string (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Revenue Growth: double (nullable = true)
 |-- Cost of Revenue: double (nullable = true)
 |-- Gross Profit: double (nullable = true)
 |-- R&D Expenses: double (nullable = true)
 |-- SG&A Expense: double (nullable = true)
 |-- Operating Expenses: double (nullable = true)
 |-- Operating Income: double (nullable = true)
 |-- Interest Expense: double (nullable = true)
 |-- Earnings before Tax: double (nullable = true)
 |-- Income Tax Expense: double (nullable = true)
 |-- Net Income - Non-Controlling int: double (nullable = true)
 |-- Net Income - Discontinued ops: double (nullable = true)
 |-- Net Income: double (nullable = true)
 |-- Preferred Dividends: double (nullable = true)
 |-- Net Income Com: double (nullable = true)
 |-- EPS: double (nullable = true)
 |-- EPS Diluted: double (nullable = true)
 |-- Weighted Average Shs Out: double (nullable = true)
 |-- Weighted Average Shs Out (Dil): double (nullab

This DataFrame has **3808 rows**.

In [4]:
stockprices.printSchema()
display(Markdown("This DataFrame has **%d rows**." % stockprices.count()))

root
 |-- date: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- Name: string (nullable = true)



This DataFrame has **619040 rows**.

## 4. Profiling the columns being considered for our analysis
### A. Important columns: Revenue, Revenue Growth, Net Profit Margin, Total liabilities, Total assets

In [5]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit, split, round, max
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import Row

print ("Summary for 2014 of columns, Revenue,Revenue Growth,Net Profit Margin,Total liabilities,Total assets")
fdata2014.select("Revenue","Revenue Growth","Net Profit Margin","Total liabilities","Total assets").summary().show()


Summary for 2014 of columns, Revenue,Revenue Growth,Net Profit Margin,Total liabilities,Total assets
+-------+--------------------+------------------+------------------+--------------------+--------------------+
|summary|             Revenue|    Revenue Growth| Net Profit Margin|   Total liabilities|        Total assets|
+-------+--------------------+------------------+------------------+--------------------+--------------------+
|  count|                3764|              3572|              3751|                3769|                3775|
|   mean| 5.879049508856397E9|12.954243701007824|-3.784724008937163|2.208639884785284...|2.642981378925977...|
| stddev|3.901740798473972E10| 705.6054728215297| 63.30058001575833|3.254539982145645E11|3.570764252078938E11|
|    min|          -6.27616E8|           -1.7732|           -3053.2|  -2.7368421052632E9|                 0.0|
|    25%|           5.78282E7|           -0.0025|           -0.0017|            5.9139E7|        1.90578612E8|
|    50%|  

### B. Top 5 stocks on the basis of different measures

In [6]:
fdatasum2014 = fdata2014.withColumnRenamed("_c0","Name")\
                        .withColumn("Debt Ratio", col("Total liabilities")/col("Total assets"))\
                        .select("Name","Revenue","Revenue Growth","Net Profit Margin","Debt Ratio")
fdatasum2015 = fdata2015.withColumnRenamed("_c0","Name")\
                        .withColumn("Debt Ratio", col("Total liabilities")/col("Total assets"))\
                        .select("Name","Revenue","Revenue Growth","Net Profit Margin","Debt Ratio")
fdatasum2016 = fdata2016.withColumnRenamed("_c0","Name")\
                        .withColumn("Debt Ratio", col("Total liabilities")/col("Total assets"))\
                        .select("Name","Revenue","Revenue Growth","Net Profit Margin","Debt Ratio")
fdatasum2017 = fdata2017.withColumnRenamed("_c0","Name")\
                        .withColumn("Debt Ratio", col("Total liabilities")/col("Total assets"))\
                        .select("Name","Revenue","Revenue Growth","Net Profit Margin","Debt Ratio")
fdatasum2018 = fdata2018.withColumnRenamed("_c0","Name")\
                        .withColumn("Debt Ratio", col("Total liabilities")/col("Total assets"))\
                        .select("Name","Revenue","Revenue Growth","Net Profit Margin","Debt Ratio")


toprev2014 = fdatasum2014.orderBy(col("Revenue Growth").desc()).select("Name","Revenue Growth")
print ("The top 5 stocks on the basis of revenue(2014):")
toprev2014.show(5)

toprev2015 = fdatasum2015.orderBy(col("Revenue Growth").desc()).select("Name","Revenue Growth")
print ("The top 5 stocks on the basis of revenue(2015):")
toprev2015.show(5)

topprof2014 = fdatasum2014.orderBy(col("Net Profit Margin").desc()).select("Name","Net Profit Margin")
print ("The top 5 stocks on the basis of Profit(2014):")
topprof2014.show(5)

topprof2015 = fdatasum2015.orderBy(col("Net Profit Margin").desc()).select("Name","Net Profit Margin")
print ("The top 5 stocks on the basis of Profit(2015):")
topprof2015.show(5)

topasset2014 = fdatasum2014.orderBy(col("Debt Ratio").desc()).select("Name","Debt Ratio")
print ("The top 5 high risk stocks on the basis of Debt Ratio(2014):")
topasset2014.show(5)

topasset2015 = fdatasum2015.orderBy(col("Debt Ratio").desc()).select("Name","Debt Ratio")
print ("The top 5 high risk stocks on the basis of Debt Ratio(2015):")
topasset2015.show(5)


The top 5 stocks on the basis of revenue(2014):
+----+--------------+
|Name|Revenue Growth|
+----+--------------+
|FPAY|    42138.6639|
|VCEL|     1514.5789|
|RDHL|         583.5|
|HEAR|      329.7432|
|ESES|      172.8504|
+----+--------------+
only showing top 5 rows

The top 5 stocks on the basis of revenue(2015):
+----+--------------+
|Name|Revenue Growth|
+----+--------------+
|NAKD|      189.8179|
|AEZS|       48.5455|
|NBLX|       41.1079|
|ONCE|       33.8045|
|AUMN|       33.3447|
+----+--------------+
only showing top 5 rows

The top 5 stocks on the basis of Profit(2014):
+----+-----------------+
|Name|Net Profit Margin|
+----+-----------------+
|  OR|          97.3961|
|SHIP|          39.9741|
|CSWC|           8.8847|
|SVVC|           6.5727|
|  KB|           4.1399|
+----+-----------------+
only showing top 5 rows

The top 5 stocks on the basis of Profit(2015):
+----+------------------+
|Name| Net Profit Margin|
+----+------------------+
|ASTC|           21.0097|
| PLX|13.2

## 5. Merging datasets: Financial dataset and actual stock
### A. Obtaining the data with date and obtain the prices after the financial data has been published

**In this case the assumption is the last working day in december of every year**


In [8]:
stockpricesSchema = StructType([StructField("date",StringType(),True)])
# Obtaining the components from date
stockswithdate = stockprices.withColumn("split_date",split("date","-"))\
                   .withColumn("year", col("split_date").getItem(0))\
                   .withColumn("month", col("split_date").getItem(1))\
                   .withColumn("dayinc", col("split_date").getItem(2))\
                   .withColumn("split_day",split("dayinc"," "))\
                   .withColumn("day", col("split_day").getItem(0))\
                   .select("day", "month", "year","Name","close")
                
stockswithdate.show(5)

+---+-----+----+----+-----+
|day|month|year|Name|close|
+---+-----+----+----+-----+
| 08|   02|2013| AAL|14.75|
| 11|   02|2013| AAL|14.46|
| 12|   02|2013| AAL|14.27|
| 13|   02|2013| AAL|14.66|
| 14|   02|2013| AAL|13.99|
+---+-----+----+----+-----+
only showing top 5 rows



In [9]:
# Obtaining the dataframe with the closing prices
stocks2013 = stockswithdate\
.where((col("day") == stockswithdate.where((col("month")=='12') & (col("year") == '2013')).select("day").rdd.max()[0]))\
.where((col("month") == '12') & (col("year") == '2013'))\
.withColumnRenamed("close","close2013")\
.withColumnRenamed("close","close2013")\
.select("Name","close2013")

stocks2014 = stockswithdate\
.where((col("day") == stockswithdate.where((col("month")=='12') & (col("year") == '2014')).select("day").rdd.max()[0]))\
.where((col("month") == '12') & (col("year") == '2014'))\
.withColumnRenamed("close","close2014")\
.select("Name","close2014")

stocks2015 = stockswithdate\
.where((col("day") == stockswithdate.where((col("month")=='12') & (col("year") == '2015')).select("day").rdd.max()[0]))\
.where((col("month") == '12') & (col("year") == '2015'))\
.withColumnRenamed("close","close2015")\
.select("Name","close2015")

stocks2016 = stockswithdate\
.where((col("day") == stockswithdate.where((col("month")=='12') & (col("year") == '2016')).select("day").rdd.max()[0]))\
.where((col("month") == '12') & (col("year") == '2016'))\
.withColumnRenamed("close","close2016")\
.select("Name","close2016")

stocks2017 = stockswithdate\
.where((col("day") == stockswithdate.where((col("month")=='12') & (col("year") == '2017')).select("day").rdd.max()[0]))\
.where((col("month") == '12') & (col("year") == '2017'))\
.withColumnRenamed("close","close2017")\
.select("Name","close2017")

#Joining the datasets split by year and obtain change over years
comp2014 = stocks2013.join(stocks2014, "Name")
comp2015 = stocks2014.join(stocks2015, "Name")
comp2016 = stocks2015.join(stocks2016, "Name")
comp2017 = stocks2016.join(stocks2017, "Name")



### B. Obtaining the increase in stock prices over years

In [10]:
# Obtaining the increment of stock prices over years

inc2014 = comp2014.withColumn("inc2014", ((col("close2014")) - col("close2013"))/(col("close2013")))\
                  .select("Name","close2013","close2014",round('inc2014', 2).alias("inc_2014"))\
                  .orderBy(col("inc2014").desc())
inc2015 = comp2015.withColumn("inc2015", ((col("close2015")) - col("close2014"))/(col("close2014")))\
                  .select("Name","close2014","close2015",round('inc2015', 2).alias("inc_2015"))\
                  .orderBy(col("inc2015").desc())
inc2016 = comp2016.withColumn("inc2016", ((col("close2016")) - col("close2015"))/(col("close2015")))\
                  .select("Name","close2015","close2016",round('inc2016', 2).alias("inc_2016"))\
                  .orderBy(col("inc2016").desc())
inc2017 = comp2017.withColumn("inc2017", ((col("close2017")) - col("close2016"))/(col("close2016")))\
                  .select("Name","close2016","close2017",round('inc2017', 2).alias("inc_2017"))\
                  .orderBy(col("inc2017").desc())
print("The top 5 stocks in terms of price increase(2014): ")   
inc2014.show(5)

print("The top 5 stocks in terms of price increase(2015): ")   
inc2015.show(5)

print("The top 5 stocks in terms of price increase(2016): ")   
inc2016.show(5)

print("The top 5 stocks in terms of price increase(2017): ")   
inc2017.show(5)

                                

The top 5 stocks in terms of price increase(2014): 
+----+---------+---------+--------+
|Name|close2013|close2014|inc_2014|
+----+---------+---------+--------+
|SWKS|    28.56|    72.71|    1.55|
| LUV|    18.84|    42.32|    1.25|
| AAL|    25.25|    53.63|    1.12|
|  EA|    22.94|   47.015|    1.05|
|  EW|    32.88|    63.69|    0.94|
+----+---------+---------+--------+
only showing top 5 rows

The top 5 stocks in terms of price increase(2015): 
+----+---------+---------+--------+
|Name|close2014|close2015|inc_2015|
+----+---------+---------+--------+
|NFLX|  48.8014|   114.38|    1.34|
|AMZN|   310.35|   675.89|    1.18|
|ATVI|    20.15|    38.71|    0.92|
| AYI|   140.07|    233.8|    0.67|
|NVDA|    20.05|    32.96|    0.64|
+----+---------+---------+--------+
only showing top 5 rows

The top 5 stocks in terms of price increase(2016): 
+----+---------+---------+--------+
|Name|close2015|close2016|inc_2016|
+----+---------+---------+--------+
| AMD|     2.87|    11.34|    2.95|
|N

### C. Developing analysis and categorized dataframes

In [11]:
# Obtaining the dataframes for our analysis
anly2014 = inc2014.join(fdatasum2014, "Name")
anly2015 = inc2015.join(fdatasum2015, "Name")
anly2016 = inc2016.join(fdatasum2016, "Name")
anly2017 = inc2017.join(fdatasum2014, "Name")

anly2014.orderBy(col("Revenue Growth").desc()).show(5)

+----+---------+---------+--------+---------+--------------+-----------------+-------------------+
|Name|close2013|close2014|inc_2014|  Revenue|Revenue Growth|Net Profit Margin|         Debt Ratio|
+----+---------+---------+--------+---------+--------------+-----------------+-------------------+
| ICE|   44.984|   43.858|   -0.03|  4.352E9|        1.5156|           0.2254| 0.8160254344067747|
|GILD|     75.1|    94.26|    0.26| 2.489E10|        1.2219|           0.4862| 0.5436475882760212|
| DVN|    61.87|    61.21|   -0.01|2.0638E10|         0.985|           0.0779| 0.4798072555641132|
|  MU|    21.75|    35.01|    0.61|1.6358E10|        0.8029|           0.1861| 0.4811741613133476|
| AGN|    168.0|   257.41|    0.53| 4.6765E9|        0.7969|          -0.3487|0.46291557678456347|
+----+---------+---------+--------+---------+--------------+-----------------+-------------------+
only showing top 5 rows



In [13]:
# Obtaining the quartiles so as to segregate the data based on their distribution

qrev14 = anly2014.approxQuantile("Revenue Growth", [0.25, 0.5, 0.75], 0.01)
qnpm14 = anly2014.approxQuantile("Net Profit Margin", [0.25, 0.5, 0.75], 0.01)
qinc14 = anly2014.approxQuantile("inc_2014", [0.25, 0.5, 0.75], 0.01)

qrev15 = anly2015.approxQuantile("Revenue Growth", [0.25, 0.5, 0.75], 0.01)
qnpm15 = anly2015.approxQuantile("Net Profit Margin", [0.25, 0.5, 0.75], 0.01)
qinc15 = anly2015.approxQuantile("inc_2015", [0.25, 0.5, 0.75], 0.01)

qrev16 = anly2016.approxQuantile("Revenue Growth", [0.25, 0.5, 0.75], 0.01)
qnpm16 = anly2016.approxQuantile("Net Profit Margin", [0.25, 0.5, 0.75], 0.01)
qinc16 = anly2016.approxQuantile("inc_2016", [0.25, 0.5, 0.75], 0.01)

qrev17 = anly2017.approxQuantile("Revenue Growth", [0.25, 0.5, 0.75], 0.01)
qnpm17 = anly2017.approxQuantile("Net Profit Margin", [0.25, 0.5, 0.75], 0.01)
qinc17 = anly2017.approxQuantile("inc_2017", [0.25, 0.5, 0.75], 0.01)

# Adding the new categories which will help answer the business questions
anlycat2014 = anly2014\
                       .withColumn("Rev_quart", when(col("Revenue Growth")<=qrev14[0],1)\
                       .when((col("Revenue Growth")>qrev14[0]) & (col("Revenue Growth")<=qrev14[1]),2)\
                       .when((col("Revenue Growth")>qrev14[1]) & (col("Revenue Growth")<=qrev14[2]),3)\
                       .otherwise(4))\
                       .withColumn("Net_Profit_quart", when(col("Net Profit Margin")<=qnpm14[0],1)\
                       .when((col("Net Profit Margin")>qnpm14[0]) & (col("Net Profit Margin")<=qnpm14[1]),2)\
                       .when((col("Net Profit Margin")>qnpm14[1]) & (col("Net Profit Margin")<=qnpm14[2]),3)\
                       .otherwise(4))\
                       .withColumn("Debt_cat", when(col("Debt Ratio")<=0.4,"1.Low Risk")\
                       .when((col("Debt Ratio")>0.4) & (col("Debt Ratio")<=0.6),"2.Moderate Risk")\
                       .otherwise("3.High Risk"))\
                       .withColumn("inc14_quart", when(col("inc_2014")<=qinc14[0],1)\
                       .when((col("inc_2014")>qinc14[0]) & (col("inc_2014")<=qinc14[1]),2)\
                       .when((col("inc_2014")>qinc14[1]) & (col("inc_2014")<=qinc14[2]),3)\
                       .otherwise(4))\
                       .select("Name","inc_2014","Rev_quart","Net_Profit_quart","Debt_cat","inc14_quart")

anlycat2015 = anly2015\
                       .withColumn("Rev_quart", when(col("Revenue Growth")<=qrev15[0],1)\
                       .when((col("Revenue Growth")>qrev15[0]) & (col("Revenue Growth")<=qrev15[1]),2)\
                       .when((col("Revenue Growth")>qrev15[1]) & (col("Revenue Growth")<=qrev15[2]),3)\
                       .otherwise(4))\
                       .withColumn("Net_Profit_quart", when(col("Net Profit Margin")<=qnpm15[0],1)\
                       .when((col("Net Profit Margin")>qnpm15[0]) & (col("Net Profit Margin")<=qnpm15[1]),2)\
                       .when((col("Net Profit Margin")>qnpm15[1]) & (col("Net Profit Margin")<=qnpm15[2]),3)\
                       .otherwise(4))\
                       .withColumn("Debt_cat", when(col("Debt Ratio")<=0.4,"1.Low Risk")\
                       .when((col("Debt Ratio")>0.4) & (col("Debt Ratio")<=0.6),"2.Moderate Risk")\
                       .otherwise("3.High Risk"))\
                       .withColumn("inc15_quart", when(col("inc_2015")<=qinc15[0],1)\
                       .when((col("inc_2015")>qinc15[0]) & (col("inc_2015")<=qinc15[1]),2)\
                       .when((col("inc_2015")>qinc15[1]) & (col("inc_2015")<=qinc15[2]),3)\
                       .otherwise(4))\
                       .select("Name","inc_2015","Rev_quart","Net_Profit_quart","Debt_cat","inc15_quart")
anlycat2016 = anly2016\
                       .withColumn("Rev_quart", when(col("Revenue Growth")<=qrev16[0],1)\
                       .when((col("Revenue Growth")>qrev16[0]) & (col("Revenue Growth")<=qrev16[1]),2)\
                       .when((col("Revenue Growth")>qrev16[1]) & (col("Revenue Growth")<=qrev16[2]),3)\
                       .otherwise(4))\
                       .withColumn("Net_Profit_quart", when(col("Net Profit Margin")<=qnpm16[0],1)\
                       .when((col("Net Profit Margin")>qnpm16[0]) & (col("Net Profit Margin")<=qnpm16[1]),2)\
                       .when((col("Net Profit Margin")>qnpm16[1]) & (col("Net Profit Margin")<=qnpm16[2]),3)\
                       .otherwise(4))\
                       .withColumn("Debt_cat", when(col("Debt Ratio")<=0.4,"1.Low Risk")\
                       .when((col("Debt Ratio")>0.4) & (col("Debt Ratio")<=0.6),"2.Moderate Risk")\
                       .otherwise("3.High Risk"))\
                       .withColumn("inc16_quart", when(col("inc_2016")<=qinc16[0],1)\
                       .when((col("inc_2016")>qinc16[0]) & (col("inc_2016")<=qinc16[1]),2)\
                       .when((col("inc_2016")>qinc16[1]) & (col("inc_2016")<=qinc16[2]),3)\
                       .otherwise(4))\
                       .select("Name","inc_2016","Rev_quart","Net_Profit_quart","Debt_cat","inc16_quart")

anlycat2017 = anly2017\
                       .withColumn("Rev_quart", when(col("Revenue Growth")<=qrev17[0],1)\
                       .when((col("Revenue Growth")>qrev17[0]) & (col("Revenue Growth")<=qrev17[1]),2)\
                       .when((col("Revenue Growth")>qrev17[1]) & (col("Revenue Growth")<=qrev17[2]),3)\
                       .otherwise(4))\
                       .withColumn("Net_Profit_quart", when(col("Net Profit Margin")<=qnpm17[0],1)\
                       .when((col("Net Profit Margin")>qnpm17[0]) & (col("Net Profit Margin")<=qnpm17[1]),2)\
                       .when((col("Net Profit Margin")>qnpm17[1]) & (col("Net Profit Margin")<=qnpm17[2]),3)\
                       .otherwise(4))\
                       .withColumn("Debt_cat", when(col("Debt Ratio")<=0.4,"1.Low Risk")\
                       .when((col("Debt Ratio")>0.4) & (col("Debt Ratio")<=0.6),"2.Moderate Risk")\
                       .otherwise("3.High Risk"))\
                       .withColumn("inc17_quart", when(col("inc_2017")<=qinc17[0],1)\
                       .when((col("inc_2017")>qinc17[0]) & (col("inc_2017")<=qinc17[1]),2)\
                       .when((col("inc_2017")>qinc17[1]) & (col("inc_2017")<=qinc17[2]),3)\
                       .otherwise(4))\
                       .select("Name","inc_2017","Rev_quart","Net_Profit_quart","Debt_cat","inc17_quart")

anlycat2014.show()


+----+--------+---------+----------------+---------------+-----------+
|Name|inc_2014|Rev_quart|Net_Profit_quart|       Debt_cat|inc14_quart|
+----+--------+---------+----------------+---------------+-----------+
|SWKS|    1.55|        4|               4|     1.Low Risk|          4|
| LUV|    1.25|        2|               1|    3.High Risk|          4|
| AAL|    1.12|        4|               2|    3.High Risk|          4|
|  EA|    1.05|        1|               1|2.Moderate Risk|          4|
|  EW|    0.94|        4|               4|     1.Low Risk|          4|
|AVGO|     0.9|        4|               1|    3.High Risk|          4|
| DAL|    0.79|        3|               1|    3.High Risk|          4|
| UAL|    0.77|        2|               1|    3.High Risk|          4|
| CNC|    0.76|        4|               1|    3.High Risk|          4|
| RCL|    0.74|        2|               2|    3.High Risk|          4|
| SIG|    0.67|        3|               2|     1.Low Risk|          4|
|ILMN|

## 6. Answer Business questions
### A. How many stocks have low and moderate risk, top quartile in revenue, net profit, and bottom two quartiles in terms of price per each category?

In [15]:
anlycat2014.groupBy("Rev_quart","Net_Profit_quart","Debt_cat","inc14_quart").agg(count(lit(1)).alias("Total"))\
            .where((col("Rev_quart") == 4) & (col("Net_Profit_quart") == 4)\
            & ((col("Debt_cat") == "1.Low Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc14_quart") == 1)|(col("inc14_quart") == 2))).show()

anlycat2015.groupBy("Rev_quart","Net_Profit_quart","Debt_cat","inc15_quart").agg(count(lit(1)).alias("Total"))\
            .where((col("Rev_quart") == 4) & (col("Net_Profit_quart") == 4)\
            & ((col("Debt_cat") == "1.Low Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc15_quart") == 1)|(col("inc15_quart") == 2))).show()

anlycat2016.groupBy("Rev_quart","Net_Profit_quart","Debt_cat","inc16_quart").agg(count(lit(1)).alias("Total"))\
            .where((col("Rev_quart") == 4) & (col("Net_Profit_quart") == 4)\
            & ((col("Debt_cat") == "1.Low Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc16_quart") == 1)|(col("inc16_quart") == 2))).show()

anlycat2017.groupBy("Rev_quart","Net_Profit_quart","Debt_cat","inc17_quart").agg(count(lit(1)).alias("Total"))\
            .where((col("Rev_quart") == 4) & (col("Net_Profit_quart") == 4)\
            & ((col("Debt_cat") == "1.Low Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc17_quart") == 1)|(col("inc17_quart") == 2))).show()

+---------+----------------+---------------+-----------+-----+
|Rev_quart|Net_Profit_quart|       Debt_cat|inc14_quart|Total|
+---------+----------------+---------------+-----------+-----+
|        4|               4|2.Moderate Risk|          1|    5|
|        4|               4|     1.Low Risk|          1|    2|
|        4|               4|2.Moderate Risk|          2|    1|
|        4|               4|     1.Low Risk|          2|    1|
+---------+----------------+---------------+-----------+-----+

+---------+----------------+---------------+-----------+-----+
|Rev_quart|Net_Profit_quart|       Debt_cat|inc15_quart|Total|
+---------+----------------+---------------+-----------+-----+
|        4|               4|2.Moderate Risk|          1|    1|
|        4|               4|2.Moderate Risk|          2|    6|
|        4|               4|     1.Low Risk|          2|    2|
+---------+----------------+---------------+-----------+-----+

+---------+----------------+---------------+---------

### B. Which stocks are low risk, best performing and a good buy?



In [16]:
print("Underpriced stocks to buy for 2014 are:")
anlycat2014.where((col("Rev_quart") == 4) & (col("Net_Profit_quart") == 4)\
            & ((col("Debt_cat") == "1.Low Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc14_quart") == 1)|(col("inc14_quart") == 2))).show()

print("Underpriced stocks to buy for 2015 are:")
anlycat2015.where((col("Rev_quart") == 4) & (col("Net_Profit_quart") == 4)\
            & ((col("Debt_cat") == "1.Low Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc15_quart") == 1)|(col("inc15_quart") == 2))).show()

print("Underpriced stocks to buy for 2016 are:")
anlycat2016.where((col("Rev_quart") == 4) & (col("Net_Profit_quart") == 4)\
            & ((col("Debt_cat") == "1.Low Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc16_quart") == 1)|(col("inc16_quart") == 2))).show()

print("Underpriced stocks to buy for 2017 are:")
anlycat2017.where((col("Rev_quart") == 4) & (col("Net_Profit_quart") == 4)\
            & ((col("Debt_cat") == "1.Low Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc17_quart") == 1)|(col("inc17_quart") == 2))).show()

Underpriced stocks to buy for 2014 are:
+-----+--------+---------+----------------+---------------+-----------+
| Name|inc_2014|Rev_quart|Net_Profit_quart|       Debt_cat|inc14_quart|
+-----+--------+---------+----------------+---------------+-----------+
|   MA|    0.03|        4|               4|2.Moderate Risk|          2|
| TROW|    0.02|        4|               4|     1.Low Risk|          2|
|  XEC|    0.01|        4|               4|2.Moderate Risk|          1|
| MCHP|    0.01|        4|               4|2.Moderate Risk|          1|
| ALGN|   -0.02|        4|               4|     1.Low Risk|          1|
|GOOGL|   -0.05|        4|               4|     1.Low Risk|          1|
|  CXO|   -0.08|        4|               4|2.Moderate Risk|          1|
| TRIP|    -0.1|        4|               4|2.Moderate Risk|          1|
|  PXD|   -0.19|        4|               4|2.Moderate Risk|          1|
+-----+--------+---------+----------------+---------------+-----------+

Underpriced stocks to b

### C. Which low to moderate risk stocks to buy to play the trend?


In [17]:
print("Trending stocks to buy for 2014 are:")
anlycat2014.where((col("Rev_quart") == 4) & (col("Net_Profit_quart") == 4)\
            & ((col("Debt_cat") == "1.Low Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc14_quart") == 3)|(col("inc14_quart") == 4))).show()

print("Trending stocks to buy for 2015 are:")
anlycat2015.where((col("Rev_quart") == 4) & (col("Net_Profit_quart") == 4)\
            & ((col("Debt_cat") == "1.Low Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc15_quart") == 3)|(col("inc15_quart") == 4))).show()

print("Trending stocks to buy for 2016 are:")
anlycat2016.where((col("Rev_quart") == 4) & (col("Net_Profit_quart") == 4)\
            & ((col("Debt_cat") == "1.Low Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc16_quart") == 3)|(col("inc16_quart") == 4))).show()

print("Trending stocks to buy for 2017 are:")
anlycat2017.where((col("Rev_quart") == 4) & (col("Net_Profit_quart") == 4)\
            & ((col("Debt_cat") == "1.Low Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc17_quart") == 3)|(col("inc17_quart") == 4))).show()

Trending stocks to buy for 2014 are:
+----+--------+---------+----------------+---------------+-----------+
|Name|inc_2014|Rev_quart|Net_Profit_quart|       Debt_cat|inc14_quart|
+----+--------+---------+----------------+---------------+-----------+
|SWKS|    1.55|        4|               4|     1.Low Risk|          4|
|  EW|    0.94|        4|               4|     1.Low Risk|          4|
|ILMN|    0.67|        4|               4|2.Moderate Risk|          4|
|  MU|    0.61|        4|               4|2.Moderate Risk|          4|
|FFIV|    0.44|        4|               4|     1.Low Risk|          4|
|  FB|    0.43|        4|               4|     1.Low Risk|          4|
|ALXN|    0.39|        4|               4|     1.Low Risk|          4|
| EXR|    0.39|        4|               4|2.Moderate Risk|          4|
| AVB|    0.38|        4|               4|2.Moderate Risk|          4|
| SLG|    0.29|        4|               4|2.Moderate Risk|          4|
| GLW|    0.29|        4|               

### D. Which stocks to avoid ?

In [14]:
print("Stocks to avoid for 2014 are:")
anlycat2014.where((col("Rev_quart") == 1) & (col("Net_Profit_quart") == 1)\
            & ((col("Debt_cat") == "3.High Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc14_quart") == 3)|(col("inc14_quart") == 4))).show()

print("Stocks to avoid for 2015 are:")
anlycat2015.where((col("Rev_quart") == 1) & (col("Net_Profit_quart") == 1)\
            & ((col("Debt_cat") == "3.High Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc15_quart") == 3)|(col("inc15_quart") == 4))).show()

print("Stocks to avoid for 2016 are:")
anlycat2016.where((col("Rev_quart") == 1) & (col("Net_Profit_quart") == 1)\
            & ((col("Debt_cat") == "3.High Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc16_quart") == 3)|(col("inc16_quart") == 4))).show()

print("Stocks to avoid for 2017 are:")
anlycat2017.where((col("Rev_quart") == 1) & (col("Net_Profit_quart") == 1)\
            & ((col("Debt_cat") == "3.High Risk")|(col("Debt_cat") == "2.Moderate Risk"))\
            & ((col("inc17_quart") == 3)|(col("inc17_quart") == 4))).show()

Stocks to avoid for 2014 are:
+----+--------+---------+----------------+---------------+-----------+
|Name|inc_2014|Rev_quart|Net_Profit_quart|       Debt_cat|inc14_quart|
+----+--------+---------+----------------+---------------+-----------+
|  EA|    1.05|        1|               1|2.Moderate Risk|          4|
|VRTX|     0.6|        1|               1|2.Moderate Risk|          4|
|ARNC|    0.49|        1|               1|    3.High Risk|          4|
|COTY|    0.35|        1|               1|    3.High Risk|          4|
| SEE|    0.25|        1|               1|    3.High Risk|          3|
| CTL|    0.24|        1|               1|    3.High Risk|          3|
|   M|    0.23|        1|               1|    3.High Risk|          3|
| CAH|    0.21|        1|               1|    3.High Risk|          3|
| TGT|     0.2|        1|               1|    3.High Risk|          3|
| ADM|     0.2|        1|               1|2.Moderate Risk|          3|
|  FE|    0.18|        1|               1|    3